In [6]:
import pandas as pd
import numpy as np



# from scipy.stats import spearmanr, rankdata
# from scipy.spatial.distance import pdist, squareform
import random
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import KFold

In [7]:
def load_data(path):

    files = os.listdir(f'{path}\\img\\data\\')
    image = np.empty((len(files), 50, 50, 1))
    sample = []
    id = []
    for i in range(len(files)):
        if files[i].split('.')[1] == 'txt':
            id.append(i)
            data = pd.read_csv(f'{path}\\img\\data\\{files[i]}', sep='\t', engine='c', na_values=['na', '-', ''], header=None, index_col=None)
            image[i, :, :, 0] = data.values
            sample.append(files[i].split('.txt')[0])
    image = image[id, :, :, :]

    return image

In [8]:
def top_n_miRNA(input_data, n):
    temp_df = input_data.copy()
    temp_df['sums'] = (temp_df.iloc[:,1:].sum(axis = 1))
    temp_df = temp_df.sort_values(by='sums', ascending=False)
    return temp_df.iloc[0:n,0:-1]


In [9]:
def balance_data(x_tn, y_lb):
    if(x_tn.shape[0]==y_lb.shape[0]):
        nonzero_index_r = y_lb.index[y_lb['amount']>0].to_list() #list of zero indices
        zero_index_r = y_lb.index[y_lb['amount']==0].to_list() #list of zero indices
        zero_index_r = random.choices(zero_index_r, k=len(nonzero_index_r))
        final_ind = nonzero_index_r + zero_index_r

        x_tn_temp = all_data[final_ind,:,:,:]
        y_lb_temp = y_lb.loc[final_ind]
        return x_tn_temp, y_lb_temp
    else:
        print('Dataset Shape did not match!!')

In [68]:
# def train_test_data(data_main, test_per = 0.2):
#     rand_test_index= np.random.randint(0, data_main.shape[0], np.int64(data_main.shape[0]*test_per))
#     test_data = data_main[rand_test_index]
#     train_data = np.delete(data_main, rand_test_index, axis = 0)

#     return train_data, test_data, rand_test_index


# train_data, test_data, test_indx = train_test_data(all_data, test_per=0.1)

In [10]:
# data_path = '..\\Results\\Table_To_Image_Conversion\\Test_1\\data'
data_path = "D:\\Research\\data\\processed_dataset" #lab

# tissue_type = 'liver'
# tissue_type = 'pancreas'
# tissue_type = 'kidney'
tissue_type = 'heart'
# tissue_type = 'brain_myeloid'


In [11]:
all_data = load_data(path = f'{data_path}\\{tissue_type}')
all_data.shape

(6002, 50, 50, 1)

In [12]:
all_data = all_data/255

In [13]:
y_target = pd.read_csv(f'{data_path}\\{tissue_type}\\miRNA_bulk.csv')
y_target = y_target.drop('Unnamed: 0',axis=1)

# y_target = y_target.iloc[:,1:].drop(y_target.index[(y_target.iloc[:,1:].sum(axis = 1) < 1000)]).T


y_target.shape

# y_target = y_target.iloc[:,1:].T
# y_target = y_target.values
# y_target

# y_target['sums'] = (y_target.iloc[:,1:].sum(axis = 1))
# y_target.sort_values(by='sums', ascending=False)
# y_target.iloc[0:50,0:-1]


(224, 6003)

In [14]:
top_miRNAs = top_n_miRNA(y_target, 1)

top_miRNAs = top_miRNAs.iloc[:,1:].T
top_miRNAs = top_miRNAs.rename(columns={top_miRNAs.columns[0]: "amount"})
sIdex = [i for i in range(top_miRNAs.shape[0])]
top_miRNAs = top_miRNAs.set_index([sIdex])
top_miRNAs


,amount
0,0
1,0
2,0
3,21
4,0
...,...
5997,0
5998,35
5999,0
6000,0


In [225]:
all_data, top_miRNAs = balance_data(all_data,top_miRNAs)
top_miRNAs_lbl = top_miRNAs.amount.values


In [22]:
top_miRNAs_lbl = top_miRNAs_lbl.reshape(-1, 1)

top_miRNAs_lbl = MinMaxScaler().fit_transform(top_miRNAs_lbl)

In [23]:
top_miRNAs_lbl


array([[0.02896552],
       [0.04413793],
       [0.12827586],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [74]:
# total_sample = top_miRNAs.shape[0]
# ex_lvl = [0 for l in range(total_sample)]
# top_miRNAs['ex_lvl'] = ex_lvl
# top_miRNAs = top_miRNAs.reset_index()
# top_miRNAs


,index,amount,ex_lvl
0,3,21,0
1,5,32,0
2,7,93,0
3,8,68,0
4,15,49,0
...,...,...,...
1583,2751,0,0
1584,4167,0,0
1585,4894,0,0
1586,3214,0,0


In [75]:
# for i in range(total_sample):
#     if(top_miRNAs['amount'][i]<10):
#         top_miRNAs['ex_lvl'][i] = 0
#     elif(top_miRNAs['amount'][i]<100):
#         top_miRNAs['ex_lvl'][i] = 1
#     else:
#         top_miRNAs['ex_lvl'][i] = 2


In [76]:
# top_miRNAs_lbl = top_miRNAs.ex_lvl.values
# encoded_array = np.zeros((top_miRNAs_lbl.size, top_miRNAs_lbl.max()+1), dtype=int)
# encoded_array[np.arange(top_miRNAs_lbl.size),top_miRNAs_lbl] = 1 
# top_miRNAs_lbl = encoded_array

In [77]:
# top_miRNAs_lbl


array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [78]:
top_miRNAs

,index,amount,ex_lvl
0,3,21,1
1,5,32,1
2,7,93,1
3,8,68,1
4,15,49,1
...,...,...,...
1583,2751,0,0
1584,4167,0,0
1585,4894,0,0
1586,3214,0,0


In [297]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaled = scaler.fit_transform(top_miRNAs.iloc[:,1:])

# top_miRNAs = preprocessing.MinMaxScaler(top_miRNAs.iloc[:,1:].T)
# top_miRNAs
# top_miRNAs = top_miRNAs.iloc[:,1:].values.T

In [79]:
# top_miRNAs = top_miRNAs.T
# bins = [0, 99, 199, 299, 399, 499, 599, 699, 799, 899, 999]
# labels = ['J', 'I', 'H', 'G', 'F', 'E', 'D','C', 'B', 'A',]

def show_grade(smple):
    bins = [0, 1, 19, 29, 39, 49, 59, 69, 79, 89, 99, 999]
    labels = ['K','J', 'I', 'H', 'G', 'F', 'E', 'D','C', 'B', 'A',]
    smple['grade'] = pd.cut(x = smple['amount'], bins = bins, labels = labels, include_lowest = True)

In [80]:
show_grade(top_miRNAs)

In [24]:
top_miRNAs.sort_values(by='amount')


,amount
3439,0
2592,0
4837,0
2863,0
5828,0
...,...
4546,465
1645,495
1350,529
226,548


In [ ]:
top_miRNAs['grade'].value_counts()


In [ ]:
quantiles  = top_miRNAs['grade'].value_counts().quantile([0.25, 0.5, 0.75])
quantiles.plot(marker='o')
plt.xlabel('Quantile')
plt.ylabel('Value')
plt.title('Quantiles')
plt.show()

In [214]:
# X_train, X_test, y_train, y_test = train_test_split(all_data, y_target, test_size=0.1, random_state=5)

# print(f'X_train: {X_train.shape}')
# print(f'X_test: {X_test.shape}')
# print(f'y_train: {y_train.shape}')
# print(f'y_test: {y_test.shape}')



None


In [217]:
# for train_indices, test_indices in kfold.split(all_data):
#     y_train, y_val = top_miRNAs.values[train_indices], top_miRNAs.values[test_indices]


In [49]:
# top_miRNAs.amount.values


array([21, 32, 93, ...,  0,  0,  0], dtype=int64)

In [99]:

# def runCNN(train_x_input, train_y_input, val_x, val_y, epoch_num = 10, batch_num = 32):
# # Define the model
#     model = keras.Sequential()

#     # Add the layers to the model
#     model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
#     model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#     model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
#     model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#     model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
#     model.add(keras.layers.BatchNormalization())

#     # Flatten the output before passing it to the fully connected layers
#     model.add(keras.layers.Flatten())

#     # Add fully connected layers
#     model.add(keras.layers.Dense(128, activation='relu'))
#     model.add(keras.layers.Dense(64, activation='relu'))
#     model.add(keras.layers.Dense(train_y_input.shape[1], activation='softmax'))

#     # Compile the model
#     model.compile(optimizer='Adam',
#                 loss='mse',
#                 metrics=['accuracy'])

#     # Print the model summary
#     print(model.summary())

#     history = model.fit(train_x_input, train_y_input, epochs=epoch_num, batch_size=batch_num, validation_data = (val_x, val_y))

#     return model, history


In [83]:
def plot_history(his, tissue_type = ''):
    plt.plot(his.history['accuracy'])
    plt.plot(his.history['val_accuracy'])
    plt.title(f'Model Accuracy: {tissue_type}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(his.history['loss'])
    plt.plot(his.history['val_loss'])
    plt.title(f'Model Loss: {tissue_type}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()


In [54]:
model = keras.Sequential()

# Add the layers to the model
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(all_data.shape[1], all_data.shape[2], all_data.shape[3])))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())


# Flatten the output before passing it to the fully connected layers
model.add(keras.layers.Flatten())

# Add fully connected layers
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1))


# Define a custom learning rate
learning_rate = 0.0001
# optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer,
            loss='mean_squared_error',
            metrics=['accuracy'])

# Print the model summary
print(model.summary())




Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_27 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 24, 24, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 22, 22, 32)        18464     
                                                                 
 batch_normalization_28 (Bat  (None, 22, 22, 32)       128       
 chNormalization)                                                
                                                      

In [89]:
# model_pan, history_pan = runCNN(train_x_input=X_train, train_y_input=y_train, epoch_num=100, val_x=X_test,  val_y=y_test)

In [100]:
# plot_history(history_pan, tissue_type=tissue_type)

In [55]:
num_folds = 5
fold_scores = []
epoch_num = 50
batch_num = 32

kfold = KFold(n_splits=num_folds, shuffle=True)

# y_label = top_miRNAs.amount.values

for train_indices, test_indices in kfold.split(all_data):
    # print('Train: %s | test: %s' % (train_indices, test_indices))


    X_train, X_val = all_data[train_indices], all_data[test_indices]
    y_train, y_val = top_miRNAs_lbl[train_indices], top_miRNAs_lbl[test_indices]

    history = model.fit(X_train, y_train, epochs=epoch_num, batch_size=batch_num, validation_data = (X_val, y_val))

    _, score = model.evaluate(X_val, y_val)
    fold_scores.append(score)


Epoch 1/50
40/40 [==============================] - 1s 26ms/step - loss: 1.2010 - accuracy: 0.4575 - val_loss: 0.0198 - val_accuracy: 0.4811
Epoch 2/50
40/40 [==============================] - 1s 16ms/step - loss: 1.0981 - accuracy: 0.4291 - val_loss: 0.0659 - val_accuracy: 0.4811
Epoch 3/50
40/40 [==============================] - 1s 15ms/step - loss: 0.7071 - accuracy: 0.4307 - val_loss: 0.0851 - val_accuracy: 0.4811
Epoch 4/50
40/40 [==============================] - 1s 15ms/step - loss: 0.6778 - accuracy: 0.4323 - val_loss: 0.0896 - val_accuracy: 0.4811
Epoch 5/50
40/40 [==============================] - 1s 15ms/step - loss: 0.4873 - accuracy: 0.4323 - val_loss: 0.0805 - val_accuracy: 0.4811
Epoch 6/50
40/40 [==============================] - 1s 16ms/step - loss: 0.6021 - accuracy: 0.4276 - val_loss: 0.0568 - val_accuracy: 0.4811
Epoch 7/50
40/40 [==============================] - 1s 15ms/step - loss: 0.5378 - accuracy: 0.4283 - val_loss: 0.0546 - val_accuracy: 0.4811
Epoch 8/50
40

In [53]:
np.mean(fold_scores)

0.49625418782234193

In [49]:
10E-6 == 0.00001

True